In [1]:
#importing the libraries
import smtplib
import pandas as pd
import requests
from bs4 import BeautifulSoup
from price_parser import Price

In [2]:
#importing the csv files
PRODUCT_URL_CSV = "/Users/swapnanildas/Downloads/Product.csv"
SAVE_TO_CSV = True
PRICES_CSV = "/Users/swapnanildas/Downloads/Prices.csv"
SEND_MAIL = True

In [3]:
#reading the csv files
def get_urls(csv_file):
    df = pd.read_csv(csv_file)
    return df

In [4]:
#scrapping the prices
def process_products(df):
    updated_products = []
    for product in df.to_dict("records"):
        html = get_response(product["url"])
        product["price"] = get_price(html)
        product["alert"] = product["price"] < product["alert_price"]
        updated_products.append(product)
    return pd.DataFrame(updated_products)

In [5]:
#sxtracting the HTML from response for each URL
def get_response(url):
    response = requests.get(url)
    return response.text

In [6]:
#extracting the price from the given HTML
def get_price(html):
    soup = BeautifulSoup(html, "lxml")
    el = soup.select_one(".price_color")
    price = Price.fromstring(el.text)
    return price.amount_float

In [7]:
#geting subject and body
def get_mail(df):
    subject = "Price Drop Alert"
    body = df[df["alert"]].to_string()
    subject_and_message = f"Subject:{subject}\n\n{body}"
    return subject_and_message

In [8]:
#sending out email alerts
def send_mail(df):
    message_text = get_mail(df)
    with smtplib.SMTP("smtp.server.address", 587) as smtp:
        smtp.starttls()
        smtp.login(mail_user, mail_pass)
        smtp.sendmail(mail_user, mail_to, message_text)

In [17]:
#main fuction to execute the entire system
def main():
    df = get_urls(PRODUCT_URL_CSV)
    df_updated = process_products(df)
    if SAVE_TO_CSV:
        df_updated.to_csv(PRICES_CSV, index=False, mode="a")
    if SEND_MAIL:
        send_mail(df_updated)